In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv("churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [4]:
from sklearn.preprocessing import LabelEncoder
label_enco=LabelEncoder()
df['Gender']=label_enco.fit_transform(df['Gender'])

In [5]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder()
data=ohe.fit_transform(df[['Geography']]).toarray()
df1=pd.DataFrame(data, columns=['France', 'Germany', 'Spain'])

In [6]:
df=pd.concat([df,df1], axis=1)

df.drop('Geography', axis=1, inplace=True)

In [55]:
import pickle
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_enco, f)

with open('one_hot_encoder.pkl', 'wb') as f:
    pickle.dump(ohe, f)



In [7]:
x=df.drop('Exited', axis=1)
y=df['Exited']

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


In [59]:
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler, f)

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [15]:
from sklearn.model_selection import GridSearchCV

In [12]:
from scikeras.wrappers import KerasClassifier

In [14]:
param_grid = {
    'neuron': [16, 32, 64],
    'layer': [1, 2, ],
    'epochs': [50, 100]
}

In [11]:
def create_model(neuron=32,layer=1):
    model=Sequential()
    model.add(Dense(neuron, activation='relu', input_shape=(x_train.shape[1],)))
    for _ in range(layer-1):
        model.add(Dense(neuron, activation='relu'))
        
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [13]:
model=KerasClassifier(neuron=32, layer=1,build_fn=create_model, verbose=1)

In [16]:
grid=GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1)
grid_result = grid.fit(x_train, y_train)
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Fitting 3 folds for each of 12 candidates, totalling 36 fits


h:\DLprj\DeeEnv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)




Epoch 1/100


250/250 [==============================] - 4s 5ms/step - loss: 0.4716 - accuracy: 0.7949
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4171 - accuracy: 0.8206
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3941 - accuracy: 0.8365
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3722 - accuracy: 0.8466
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3594 - accuracy: 0.8528
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3524 - accuracy: 0.8589
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3477 - accuracy: 0.8580
Epoch 8/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3452 - accuracy: 0.8583
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3426 - accuracy: 0.8614
Epoch 10/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3406 - ac